In [53]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import plotly.graph_objects as go
from policyengine_core.charts import format_fig
import os

In [54]:
# Define the reforms
reforms = {
    "Serial phase-in, without refundabiity cap": Reform.from_dict({
        "gov.contrib.congress.wyden_smith.per_child_actc_phase_in": {
            "2024-01-01.2100-12-31": True
        },
        "gov.irs.credits.ctc.refundable.individual_max": {
            "2024-01-01.2100-12-31": 1000000
        },
    }, country_id="us"),
    
    "Serial phase-in at the first dollar": Reform.from_dict({
        "gov.contrib.congress.wyden_smith.per_child_actc_phase_in": {
            "2024-01-01.2100-12-31": True
        },
        "gov.irs.credits.ctc.refundable.phase_in.threshold": {
            "2024-01-01.2100-12-31": 0
        },
    }, country_id="us"),
    
    "Stacked phase-in at the first dollar": Reform.from_dict({
        "gov.irs.credits.ctc.refundable.phase_in.threshold": {
            "2024-01-01.2100-12-31": 0
        },
    }, country_id="us"),
    
    "Stacked phase-in, without refundability cap": Reform.from_dict({
        "gov.irs.credits.ctc.refundable.individual_max": {
            "2024-01-01.2100-12-31": 1000000
        },
    }, country_id="us")
}


In [55]:
def calculate_budgetary_impact(reform, year):
    baseline = Microsimulation(dataset="enhanced_cps_2024")
    baseline_income = baseline.calculate("household_net_income", period=year).sum()
    
    reformed = Microsimulation(reform=reform, dataset="enhanced_cps_2024")
    reformed_income = reformed.calculate("household_net_income", period=year).sum()
    
    impact = reformed_income - baseline_income
    return impact / 1e9  

In [56]:
# Create a DataFrame to store results
years = range(2024, 2034)
df = pd.DataFrame(index=years, columns=reforms.keys())


In [57]:
# Check if a progress file exists and load it
progress_file = "ctc_reforms_progress.csv"
if os.path.exists(progress_file):
    df = pd.read_csv(progress_file, index_col=0)
    print("Loaded existing progress.")
else:
    print("Starting new calculation.")

Loaded existing progress.


In [58]:
# Calculate the budgetary impact for 10 years for each reform
for year in years:
    print(f"Computing impacts for year {year}")
    for reform_name, reform in reforms.items():
        if pd.isna(df.loc[year, reform_name]):
            impact = calculate_budgetary_impact(reform, year)
            df.loc[year, reform_name] = impact
            print(f"  {reform_name}: ${impact:.2f} billion")
            
            # Save progress after each calculation
            df.to_csv(progress_file)
            print(f"  Progress saved to {progress_file}")
        else:
            print(f"  {reform_name}: Already computed. Skipping.")


Computing impacts for year 2024
  Serial phase-in, without refundabiity cap: Already computed. Skipping.
  Serial phase-in at the first dollar: Already computed. Skipping.
  Stacked phase-in at the first dollar: Already computed. Skipping.
  Stacked phase-in, without refundability cap: Already computed. Skipping.
Computing impacts for year 2025
  Serial phase-in, without refundabiity cap: Already computed. Skipping.
  Serial phase-in at the first dollar: Already computed. Skipping.
  Stacked phase-in at the first dollar: Already computed. Skipping.
  Stacked phase-in, without refundability cap: Already computed. Skipping.
Computing impacts for year 2026
  Serial phase-in, without refundabiity cap: Already computed. Skipping.
  Serial phase-in at the first dollar: Already computed. Skipping.
  Stacked phase-in at the first dollar: Already computed. Skipping.
  Stacked phase-in, without refundability cap: Already computed. Skipping.
Computing impacts for year 2027
  Serial phase-in, with

In [59]:
# Calculate the total 10-year impact for each reform
total_impacts = df.sum()

In [60]:
# Create a bar chart for yearly impacts
fig1 = go.Figure()

colors = ['#17354F', '#2C6496', '#1d3e5e', '#D2D2D2']

In [61]:
for i, column in enumerate(df.columns):
    fig1.add_trace(go.Bar(
        x=df.index,
        y=df[column],
        name=column,
        marker_color=colors[i],
        text=[f'${impact:.2f}B' for impact in df[column]],
        textposition='outside'
    ))

fig1.update_layout(
    title="Yearly Budgetary Impact of CTC Reforms (2024-2033)",
    xaxis_title="Year",
    yaxis_title="Budgetary Impact (Billions $)",
    barmode='group',
    legend_title="Reform",
    xaxis=dict(tickmode='linear', tick0=2024, dtick=1, tickangle=45)
)

fig1 = format_fig(fig1)

# Create a bar chart for total 10-year impacts
fig2 = go.Figure(go.Bar(
    x=total_impacts.index,
    y=total_impacts.values,
    marker_color=colors,
    text=[f'${impact:.2f}B' for impact in total_impacts.values],
    textposition='outside'
))

fig2.update_layout(
    title="Total 10-Year Budgetary Impact of CTC Reforms (2024-2033)",
    xaxis_title="Reform",
    yaxis_title="Total Budgetary Impact (Billions $)",
    xaxis=dict(tickangle=45)
)

fig2 = format_fig(fig2)

# Display the charts
fig1.show()
fig2.show()